# IEORE4004: OPTIMIZATION MODELS AND METHODS
### Created by Yuri Faenza on 09/15/24

# Formulation and solution with Gurobi of a Cash Flow problem

First formulation

In [ ]:
# Gurobipy is a library for mathematical programming available for Python
# More info in: https://www.gurobi.com/documentation/current/refman/py_python_api_overview.html


# We can install a free limited version on Collab with the following command

!pip install gurobipy
import gurobipy as gp

In [ ]:
# We can initialize a model as follows

m = gp.Model()

In [ ]:
# Add variables

xa = m.addVar(name='xa')
xb = m.addVar(name='xb')
xc = m.addVar(name='xc')
y0 = m.addVar(name='y0')
y1 = m.addVar(name='y1')
y2 = m.addVar(name='y2')
y3 = m.addVar(name='y3')

In [ ]:
# Add constraints

C0 = m.addConstr(y0 - 100 + xa + xb == 0)
C1 = m.addConstr(y1 - 0.1 * xa - 0.2 * xb - 1.02 * y0 + xc == 0)
C2 = m.addConstr(y2 - 1.1 * xb - 1.02 * y1 == 0)
C3 = m.addConstr(y3 - 1.3 * xa - 1.5 * xc - 1.02 * y2 == 0)

nxa = m.addConstr (xa >= 0)
nxb = m.addConstr (xb >= 0)
nxb = m.addConstr (xc >= 0)
ny0 = m.addConstr (y0 >= 0)
ny1 = m.addConstr (y1 >= 0)
ny2 = m.addConstr (y2 >= 0)
ny3 = m.addConstr (y3 >= 0)



In [ ]:
# Set objective function

m.setObjective(y3, gp.GRB.MAXIMIZE)

In [ ]:
# Find optimal solution

m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 11 rows, 7 columns and 22 nonzeros
Coefficient statistics:
  Matrix range     [1e-01, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0713040e+30   1.286960e+31   1.071304e+00      0s
       1    1.5300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.530000000e+02


In [ ]:
# Recover variable values on optimal

print(xa.X, xb.X, xc.X, )

print(y3.X)

0.0 0.0 102.0
153.0


The general formulation

In [ ]:
# Let's create and solve a random instance of the Cash Flow Problem

n = 15 # 15 investments
T = 20 # Horizon of 20 years
r = 0.02 # return of the money market account
b = 1000 # initial budget

# We create randomly the matrix Q corresponding to the Cash Flows

from random import randint

Q = []

R = []
for j in range(n):
  R.append(randint(-10,-1))

Q.append(R)

for t in range(1,T+1):

    # We create a list for the t-th row
    R = []
    for j in range(n):
        R.append(randint(-30, 50))

    # Add the new row to the matrix
    Q.append(R)

Q



[[-9, -8, -2, -10, -2, -3, -10, -2, -7, -1, -2, -8, -2, -7, -2],
 [-22, -17, 48, 12, -15, 27, 42, 1, 29, -11, 4, -29, 24, 6, 10],
 [16, 6, 50, 18, -3, 9, -19, -10, -9, 26, 50, 36, -11, 14, 45],
 [50, 4, 48, 42, 45, -10, -28, 28, -21, -18, 31, -15, 21, 34, -8],
 [7, -1, 38, -18, -13, 8, 31, -18, -23, 32, -3, -19, 37, 38, 17],
 [13, 17, 45, 21, -21, 11, 7, 9, -19, 25, -11, -19, -16, 44, -10],
 [-13, -30, -10, 7, -7, 4, 40, 15, 7, 3, -3, 3, -6, -3, 45],
 [9, 36, 0, -16, -9, 2, 17, 36, 46, 8, 5, -1, 17, 31, 8],
 [-21, 24, 4, 19, -17, 24, -2, -5, 41, 37, -9, 36, 8, -15, -12],
 [-18, 7, 47, -3, 41, -16, -26, -13, -24, 38, 4, -29, 12, -10, 13],
 [-2, -2, 20, 34, -30, -29, 42, -22, -5, 14, 12, 27, -28, -2, 38],
 [-28, -29, 24, -3, -23, 3, 21, 24, 3, 8, -7, -11, 9, 36, 1],
 [23, -21, 36, 40, 23, -28, -14, -22, 39, 25, -6, 37, 26, -2, 25],
 [-24, 35, 34, -27, 48, 36, 9, 48, 28, 0, 16, 20, 22, 9, -9],
 [-23, 12, -8, -1, -3, -19, -29, 39, 10, -12, -5, 40, 5, 38, 35],
 [-1, -23, 2, 26, 11, -29, -15

In [ ]:
# Model in gurobi

m = gp.Model()

x = m.addVars(range(n))
y = m.addVars(range(T+1))

# Constraints for y0
m.addConstr(y[0] - sum(Q[0][j] * x[j] for j in range(n)) - b == 0)

# Constraints y1, ... yt
for t in range(1,T+1):
    m.addConstr(y[t] - sum(Q[t][j] * x[j] for j in range(n)) - (1+r) * y[t-1] == 0)

# Nonnegativity Constraints
for t in range(T+1):
  m.addConstr(y[t] >= 0)
for j in range(n):
  m.addConstr(x[j] >= 0)

m.setObjective(y[T], gp.GRB.MAXIMIZE)

m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 57 rows, 36 columns and 390 nonzeros
Model fingerprint: 0xa93bec4e
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+03]
Presolve removed 36 rows and 0 columns
Presolve time: 0.01s
Presolved: 21 rows, 36 columns, 354 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8933188e+06   1.115309e+05   0.000000e+00      0s
       6    2.6161227e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.616122734e+05


In [ ]:
for j in range(n):
  print (x[j].X)

0.0
0.0
157.14285714285714
0.0
0.0
0.0
0.0
0.0
0.0
685.7142857142858
0.0
0.0
0.0
0.0
0.0


In [ ]:
for t in range(T+1):
  print(y[t].X)

0.0
0.0
25685.71428571429
21399.428571428572
49741.70285714286
74950.82262857143
76935.55336685716
83959.9787199086
111639.17829430677
147314.81900305004
163003.9725259682
175521.19483363043
201831.61873030305
211211.10824776624
205949.6161270073
228897.17987811886
232746.5520471098
261730.05451662344
264721.79846409877
269087.66300480935
261612.27340776267
